# CityBikes

Need to add the relevant libraries first


In [2]:
import requests
import os
import pandas as pd

Explore the structure of the API, query the API and understand the data returned.


In [3]:
# API End points
# http://api.citybik.es/v2/networks
# {
#   "networks": [
#     {
#         "company": "JCDecaux", 
#         "href": "/v2/networks/velib", <--- network API endpoint
#         "location": {
#           "latitude": 48.856612, 
#           "city": "Paris", 
#           "longitude": 2.352233, 
#           "country": "FRA"
#         }, 
#         "name": "Vélib'", 
#         "id": "velib"
#     },
#     {...}
#   ]
# }
# http://api.citybik.es/v2/networks/network_id

# {
#   "network": {
#     "name": "Vélib'", 
#     "stations": [
#       {
#           "name": "00903 - QUAI MAURIAC  / PONT DE BERCY",   /   UTC Zulu timestamp of the last time
#           "timestamp": "2014-04-14T12:10:17.622Z",  <-------/ the station was updated on our systems
#           "longitude": 2.374340554605615,                       
#           "free_bikes": 1,   <-------------------------- Available bikes
#           "latitude": 48.83713368945151, 
#           "empty_slots": 19,  <------------------------- Empty spaces
#           "id": "f5a551a87eec15155d6409fe9d0ff8e2" <---- Unique id for this station
#       },
#       {...}
#     ], 
#     "company": "JCDecaux",           |
#     "href": "/v2/networks/velib",       |        Redundant Information
#     "location": {                    |
#       "latitude": 48.856612,         |    Just so you know where you are
#       "city": "Paris",               |---      with less requests
#       "longitude": 2.352233,         | 
#       "country": "FRA"               |    This can be filtered by passing
#     },                               |      ?fields=stations to the URI
#     "id": "velib"                    |
#   }
# }


Send a request to CityBikes for the city of your choice. 


In [4]:
# Step 1: Choose a city and get the network ID from the /v2/networks endpoint
city_name = 'Barcelona'
networks_url = 'http://api.citybik.es/v2/networks'
response = requests.get(networks_url)
networks_data = response.json()

In [6]:
# Find the network information for the chosen city
network_info = next((network for network in networks_data['networks'] if network['location']['city'] == city_name), None)

if network_info:
    # Step 2: Use the network ID to retrieve information about bike stations in that city
    network_id = network_info['id']
    api_url = f'http://api.citybik.es/v2/networks/{network_id}'

    response = requests.get(api_url)
    data = response.json()

In [7]:
#exploring how response work
print(type(data))
print(type(data['network']['stations']))
data['network']['stations'][0].keys()

<class 'dict'>
<class 'list'>


dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [8]:
data['network']['stations'][10]

{'empty_slots': 17,
 'extra': {'ebikes': 1,
  'has_ebikes': True,
  'normal_bikes': 3,
  'online': True,
  'uid': 103},
 'free_bikes': 4,
 'id': 'de0e8ba6e1c44e2b7c452a639777e19a',
 'latitude': 41.410098,
 'longitude': 2.1884487,
 'name': 'C/ ARAGÓ, 629',
 'timestamp': '2023-12-09T18:28:23.417000Z'}

In [9]:
# Step 3: Extract relevant station information
station_data = []
for station in data['network']['stations']:
        station_info = {
            'latitude': station['latitude'],
            'longitude': station['longitude'],
            'num_bikes': station['free_bikes'],
            
        }
        station_data.append(station_info)

Put your parsed results into a DataFrame.

In [10]:
    # Step 4: Create a Pandas DataFrame
    df = pd.DataFrame(station_data)


In [11]:
 # Step 5: Explore and Analyze the Data
    # Display the first few rows of the DataFrame
print(df.head())

    # Basic statistics
print(df.describe())


    latitude  longitude  num_bikes
0  41.425364   2.185207          8
1  41.430000   2.190246          8
2  41.443365   2.190630         16
3  41.377532   2.170732         16
4  41.375116   2.152309          3
         latitude   longitude   num_bikes
count  505.000000  505.000000  505.000000
mean    41.399749    2.168005    9.275248
std      0.020970    0.024676    6.785948
min     41.346775    2.109154    0.000000
25%     41.383904    2.149055    4.000000
50%     41.396141    2.170650    8.000000
75%     41.412045    2.187244   14.000000
max     41.462095    2.220691   31.000000


In [12]:
response_data = data['network']['stations']
bike_stations = pd.json_normalize(response_data)

# drop unnecessary columns
columns_to_drop = list(bike_stations.columns[6:])
bike_stations = bike_stations.drop(columns=columns_to_drop)

In [13]:
bike_stations.head()

,empty_slots,free_bikes,id,latitude,longitude,name
0,23,8,dedcd93fbe85671e73c79baf58bd7fed,41.425364,2.185207,"C/ CARDENAL TEDESCHINI, 13"
1,15,8,5c10c2dba1c5a8e8d9dcbb251fd8eab7,41.430000,2.190246,"RAMBLA DE L'ONZE DE SETEMBRE, 69"
2,8,16,f1aaf083aaa04498aad975bc45bdc089,41.443365,2.190630,"PG. TORRAS I BAGES, 129"
3,2,16,4a30020e6926a2ba39f88be0f6bba5cb,41.377532,2.170732,"C/ SANT OLEGUER, 2"
4,21,3,e964297fbcd2fc96bb6632e5a530d0df,41.375116,2.152309,"AV. PARAL.LEL, 194"


In [14]:
bike_stations.shape

(505, 6)

In [15]:
bike_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  505 non-null    int64  
 1   free_bikes   505 non-null    int64  
 2   id           505 non-null    object 
 3   latitude     505 non-null    float64
 4   longitude    505 non-null    float64
 5   name         505 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 23.8+ KB


In [16]:
#save data to use in another notebook 
bike_stations.to_csv('../data/bike_stations_barcelona.csv')